<a href="https://colab.research.google.com/github/pavi-ninjaac/NuralNetwork_scratch/blob/main/FullyConnectedLayer/FullyConnectedLayer_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
#gonna train the XOR gate from scratch
import numpy as np

In [49]:
class FClayer:
  def __init__(self , input_size , output_size):
    self.input = None
    self.output = None
    #rondomly initialize the weights
    self.weights = np.random.rand(input_size , output_size) - 0.5
    self.bias =  np.random.rand(1,output_size) - 0.5 

  def forward_propogation(self , input):
    self.input = input
    #print('weights',self.weights)
    #print('bias',self.bias)
    self.output = np.dot(self.input , self.weights) + self.bias
    return self.output

  def back_propogation(self , output_error , learning_rate):
    input_error = np.dot(output_error , self.weights.T)
    #print('back weight' , self.weights)
    weights_error = np.dot(self.input.T , output_error)
    bias_error = output_error
    #update the weights and bias
    self.weights -= learning_rate *  weights_error
    self.bias -= learning_rate * bias_error

    return input_error

In [43]:
#activation layer
#inherite the fully connected layer



class ActivationLayer(FClayer):
  def __init__(self, actvation , activation_derivated):
    self.actvation = actvation
    self.activation_derivated = activation_derivated

  def forward_propogation(self , input_data):
    self.input = input_data
    self.output = self.actvation(self.input)
    #print('activation' , self.output)
    return self.output

  def back_propogation(self ,  output_error , learning_rate):
    return self.activation_derivated(self.input) * output_error

In [44]:
def tanh(x):
  return np.tanh(x)

def tanh_derivated(x):
  return 1-np.tanh(x) ** 2

In [45]:
#defining the lose

def mse(y_true , y_pred):
  return np.mean(np.power(y_true - y_pred , 2))

def mse_derivated(y_true , y_pred):
  return 2*(y_pred - y_true) / y_true.size

In [62]:
#defning the network or model

class model:
  def __init__(self):
    self.layers = []
    self.loss = None
    self.loss_derivated = None

  #adding layer to the model
  def add(self , layer):
    self.layers.append(layer)

  def setloss(self, loss , loss_derivated):
    self.loss = loss
    self.loss_derivated = loss_derivated

  #predict the unknow data
  def predict(self , Newinput):
    samples = len(Newinput)
    result = []
    for i in range(samples):
      output = Newinput[i]
      
      for layer in self.layers:
        output = layer.forward_propogation(output)

      result.append(output)
    return result
  
  def fit(self , x_train , y_train , epoches , learning_rate):
    sampels = len(x_train)
    for i in range(epoches):
      err = 0
      for j in range(sampels):
        output = x_train[j]
        #print(output , j) 
        for layer in self.layers:
          #print(layer)
          output = layer.forward_propogation(output)
          #print(output)
        
        #compute the loss
        err += self.loss(y_train[j] , output)

        #backpropogation
        error = self.loss_derivated(y_train[j] , output)

        #update the weights
        for layer in reversed(self.layers):
          error = layer.back_propogation(error , learning_rate)
        
    #calculate the average error and display that
      err /= sampels
      print('epoches..'+str(i)+'/'+str(epoches)+'-------'+'Error',err)


In [47]:
#lets try with XOR operation to check it works fine
X_train = np.array([[[0,0]] , [[0,1]] , [[1,0]] , [[1,1]]])
y_train = np.array([[[0]] , [[1]] , [[1]] , [[0]]])


In [63]:
#build the network
model = model()
model.add(FClayer(2,3))
model.add(ActivationLayer(tanh,tanh_derivated))
model.add(FClayer(3,1))
model.add(ActivationLayer(tanh,tanh_derivated))

model.setloss(mse , mse_derivated)
model.fit(X_train , y_train , epoches = 100 , learning_rate = 0.1)



epoches..0/100-------Error 0.360164386078472
epoches..1/100-------Error 0.29212819495295106
epoches..2/100-------Error 0.2840265248360428
epoches..3/100-------Error 0.2817389961238408
epoches..4/100-------Error 0.28046958188007665
epoches..5/100-------Error 0.2794365250697789
epoches..6/100-------Error 0.2784601032321391
epoches..7/100-------Error 0.2774879181196721
epoches..8/100-------Error 0.2765019461910285
epoches..9/100-------Error 0.2754954447589436
epoches..10/100-------Error 0.2744659730665751
epoches..11/100-------Error 0.27341294339819205
epoches..12/100-------Error 0.2723366743309427
epoches..13/100-------Error 0.2712380029659295
epoches..14/100-------Error 0.270118119038941
epoches..15/100-------Error 0.26897848926224455
epoches..16/100-------Error 0.2678208178839935
epoches..17/100-------Error 0.2666470207354268
epoches..18/100-------Error 0.2654592030966035
epoches..19/100-------Error 0.2642596372928131
epoches..20/100-------Error 0.26305073838308746
epoches..21/100-----

In [64]:
y_pred = model.predict(X_train)
y_pred

[array([[0.07107528]]),
 array([[0.66072806]]),
 array([[0.65371965]]),
 array([[0.71388975]])]

In [65]:
#almost correct more epoches will bring better than this